# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 7 2022 3:12PM,245891,19,GP09092022,"Granules Pharmaceuticals, Inc.",Released
1,Sep 7 2022 3:01PM,245883,20,ALU0007830,Alumier Labs Inc.,Released
2,Sep 7 2022 3:01PM,245883,20,ALUR576284761,Alumier Labs Inc.,Released
3,Sep 7 2022 3:01PM,245883,20,ALUR700505191,Alumier Labs Inc.,Released
4,Sep 7 2022 3:01PM,245883,20,ALUR963677234,Alumier Labs Inc.,Released
5,Sep 7 2022 3:01PM,245883,20,ALUR202117028,Alumier Labs Inc.,Released
6,Sep 7 2022 3:01PM,245883,20,ALUR909325194,Alumier Labs Inc.,Released
7,Sep 7 2022 3:01PM,245883,20,ALU0007834,Alumier Labs Inc.,Released
8,Sep 7 2022 3:01PM,245883,20,ALUR169541059,Alumier Labs Inc.,Released
9,Sep 7 2022 3:01PM,245883,20,ALUR516934081,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,245885,Released,17
45,245887,Released,1
46,245888,Released,1
47,245889,Released,1
48,245891,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245885,NaN,NaN,17.0
245887,NaN,NaN,1.0
245888,NaN,NaN,1.0
245889,NaN,NaN,1.0
245891,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245714,22.0,0.0,4.0
245786,7.0,2.0,2.0
245794,0.0,1.0,0.0
245807,0.0,0.0,19.0
245808,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245714,22,0,4
245786,7,2,2
245794,0,1,0
245807,0,0,19
245808,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245714,22,0,4
1,245786,7,2,2
2,245794,0,1,0
3,245807,0,0,19
4,245808,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245714,22,,4
1,245786,7,2,2
2,245794,,1,
3,245807,,,19
4,245808,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 7 2022 3:12PM,245891,19,"Granules Pharmaceuticals, Inc."
1,Sep 7 2022 3:01PM,245883,20,Alumier Labs Inc.
10,Sep 7 2022 2:58PM,245889,20,"Adare Pharmaceuticals, Inc."
11,Sep 7 2022 2:47PM,245888,19,ACG North America LLC
12,Sep 7 2022 2:44PM,245887,10,ISDIN Corporation
13,Sep 7 2022 2:44PM,245885,10,ISDIN Corporation
30,Sep 7 2022 2:33PM,245884,10,Emerginnova
32,Sep 7 2022 2:22PM,245879,19,"AdvaGen Pharma, Ltd"
34,Sep 7 2022 2:09PM,245873,10,ISDIN Corporation
65,Sep 7 2022 1:59PM,245877,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 7 2022 3:12PM,245891,19,"Granules Pharmaceuticals, Inc.",,,1
1,Sep 7 2022 3:01PM,245883,20,Alumier Labs Inc.,,,9
2,Sep 7 2022 2:58PM,245889,20,"Adare Pharmaceuticals, Inc.",,,1
3,Sep 7 2022 2:47PM,245888,19,ACG North America LLC,,,1
4,Sep 7 2022 2:44PM,245887,10,ISDIN Corporation,,,1
5,Sep 7 2022 2:44PM,245885,10,ISDIN Corporation,,,17
6,Sep 7 2022 2:33PM,245884,10,Emerginnova,,,2
7,Sep 7 2022 2:22PM,245879,19,"AdvaGen Pharma, Ltd",,,2
8,Sep 7 2022 2:09PM,245873,10,ISDIN Corporation,,,31
9,Sep 7 2022 1:59PM,245877,10,Emerginnova,,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 7 2022 3:12PM,245891,19,"Granules Pharmaceuticals, Inc.",1,,
1,Sep 7 2022 3:01PM,245883,20,Alumier Labs Inc.,9,,
2,Sep 7 2022 2:58PM,245889,20,"Adare Pharmaceuticals, Inc.",1,,
3,Sep 7 2022 2:47PM,245888,19,ACG North America LLC,1,,
4,Sep 7 2022 2:44PM,245887,10,ISDIN Corporation,1,,
5,Sep 7 2022 2:44PM,245885,10,ISDIN Corporation,17,,
6,Sep 7 2022 2:33PM,245884,10,Emerginnova,2,,
7,Sep 7 2022 2:22PM,245879,19,"AdvaGen Pharma, Ltd",2,,
8,Sep 7 2022 2:09PM,245873,10,ISDIN Corporation,31,,
9,Sep 7 2022 1:59PM,245877,10,Emerginnova,11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 7 2022 3:12PM,245891,19,"Granules Pharmaceuticals, Inc.",1,,
1,Sep 7 2022 3:01PM,245883,20,Alumier Labs Inc.,9,,
2,Sep 7 2022 2:58PM,245889,20,"Adare Pharmaceuticals, Inc.",1,,
3,Sep 7 2022 2:47PM,245888,19,ACG North America LLC,1,,
4,Sep 7 2022 2:44PM,245887,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 7 2022 3:12PM,245891,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
1,Sep 7 2022 3:01PM,245883,20,Alumier Labs Inc.,9.0,NaN,NaN
2,Sep 7 2022 2:58PM,245889,20,"Adare Pharmaceuticals, Inc.",1.0,NaN,NaN
3,Sep 7 2022 2:47PM,245888,19,ACG North America LLC,1.0,NaN,NaN
4,Sep 7 2022 2:44PM,245887,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 7 2022 3:12PM,245891,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
1,Sep 7 2022 3:01PM,245883,20,Alumier Labs Inc.,9.0,0.0,0.0
2,Sep 7 2022 2:58PM,245889,20,"Adare Pharmaceuticals, Inc.",1.0,0.0,0.0
3,Sep 7 2022 2:47PM,245888,19,ACG North America LLC,1.0,0.0,0.0
4,Sep 7 2022 2:44PM,245887,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5162690,320.0,4.0,7.0
12,491695,1.0,1.0,0.0
15,983214,72.0,8.0,57.0
16,245845,2.0,0.0,0.0
19,1475272,14.0,25.0,0.0
20,983495,25.0,15.0,3.0
21,245794,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5162690,320.0,4.0,7.0
1,12,491695,1.0,1.0,0.0
2,15,983214,72.0,8.0,57.0
3,16,245845,2.0,0.0,0.0
4,19,1475272,14.0,25.0,0.0
5,20,983495,25.0,15.0,3.0
6,21,245794,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,320.0,4.0,7.0
1,12,1.0,1.0,0.0
2,15,72.0,8.0,57.0
3,16,2.0,0.0,0.0
4,19,14.0,25.0,0.0
5,20,25.0,15.0,3.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,320.0
1,12,Released,1.0
2,15,Released,72.0
3,16,Released,2.0
4,19,Released,14.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,7.0,0.0,57.0,0.0,0.0,3.0,0.0
Executing,4.0,1.0,8.0,0.0,25.0,15.0,1.0
Released,320.0,1.0,72.0,2.0,14.0,25.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,7.0,0.0,57.0,0.0,0.0,3.0,0.0
1,Executing,4.0,1.0,8.0,0.0,25.0,15.0,1.0
2,Released,320.0,1.0,72.0,2.0,14.0,25.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,7.0,0.0,57.0,0.0,0.0,3.0,0.0
1,Executing,4.0,1.0,8.0,0.0,25.0,15.0,1.0
2,Released,320.0,1.0,72.0,2.0,14.0,25.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()